# Projet électif python: Comparaison des statitstiques des équipes de NBA

Dans ce notebook, nous allons nous intéresser plus en détails aux équipes et étudier les résultats lorsqu'elles s'affrontent. Nous utiliserons le même jeu de données que lors de la comparaion des statistiques des joueurs (saison 2022-2023).

In [64]:
#%pip install pandas numpy plotly requests

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import requests

#### Récupération des données et initialisation

Tout d'abord, on crée une fonction qui permet de demander à l'utilisateur de saisir le nom d'un équipe et de la reconnaître dans sa forme abrégée. On récupère alors les noms des joueurs faisant partie de cette équipe.

In [65]:
correspondance_equipes = {
    'Phi': 'Philadelphia 76ers',
    'Dal': 'Dallas Mavericks',
    'Por': 'Portland Trail Blazers',
    'Okc': 'Oklahoma City Thunder',
    'Mil': 'Milwaukee Bucks',
    'Bos': 'Boston Celtics',
    'Bro': 'Brooklyn Nets',
    'Gol': 'Golden State Warriors',
    'Lal': 'Los Angeles Lakers',
    'Cle': 'Cleveland Cavaliers',
    'Pho': 'Phoenix Suns',
    'Mem': 'Memphis Grizzlies',
    'Atl': 'Atlanta Hawks',
    'Nor': 'New Orleans Pelicans',
    'Uta': 'Utah Jazz',
    'Nyk': 'New York Knicks',
    'Sac': 'Sacramento Kings',
    'Chi': 'Chicago Bulls',
    'Min': 'Minnesota Timberwolves',
    'Den': 'Denver Nuggets',
    'Tor': 'Toronto Raptors',
    'Lac': 'Los Angeles Clippers',
    'Cha': 'Charlotte Hornets',
    'Was': 'Washington Wizards',
    'Mia': 'Miami Heat',
    'Hou': 'Houston Rockets',
    'San': 'San Antonio Spurs',
    'Det': 'Detroit Pistons',
    'Ind': 'Indiana Pacers',
    'Orl': 'Orlando Magic'
}

def trouver_equipe(input_nom, equipes_disponibles):
    """
    Cette fonction recherche le meilleur résultat correspondant à l'input_nom parmi les noms d'équipe disponibles.
    """
    meilleur_resultat = None
    meilleur_score = 0
    
    # Convertir l'input en minuscules pour la recherche sans distinguer la casse
    input_nom = input_nom.lower()
    
    # Parcourir les noms d'équipe disponibles
    for equipe_abrege in equipes_disponibles:
        equipe_complet = correspondance_equipes.get(equipe_abrege)
        if not equipe_complet:
            continue
        
        # Convertir le nom de l'équipe en minuscules pour la comparaison sans distinguer la casse
        equipe_lower = equipe_complet.lower()
        
        # Calculer le score de correspondance entre l'input et le nom de l'équipe
        score = 0
        for mot in input_nom.split():
            if mot in equipe_lower:
                score += 1
        
        # Mettre à jour le meilleur résultat
        if score > meilleur_score:
            meilleur_score = score
            meilleur_resultat = equipe_abrege
    
    return meilleur_resultat


def liste_equipes(dataframe):
    # Récupérer les noms uniques des équipes dans la colonne "Equipe"
    equipes = dataframe['TEAM'].unique().tolist()
    return equipes

def joueurs_par_equipe(dataframe, nom_equipe):
    # Filtrer les données pour n'inclure que les joueurs de l'équipe spécifiée
    joueurs_equipe = dataframe[dataframe['TEAM'] == nom_equipe]
    return joueurs_equipe


On crée une fonction qui permet de récupérer le cinq majeur de l'équipe étudiée, c'est-à-dire les cinq joueurs qui passent le plus de temps sur le terrain et qui ont donc le plus grand impact sur la réussite de leur équipe.

In [66]:
chemin_equipe_csv = "NBA Stats 202223 All Stats  NBA Player Props Tool.csv"
# Importer les données depuis le fichier team CSV
team_df = pd.read_csv(chemin_equipe_csv)
attributs = ['PPG', 'RPG', 'APG', 'BPG', 'SPG', 'TPG', 'TS%']

equipes_disponibles = liste_equipes(team_df)
equipe1 = input("Entrez le nom de l'équipe : ")
equipe_trouvee = trouver_equipe(equipe1, equipes_disponibles)
joueurs1 = joueurs_par_equipe(team_df, equipe_trouvee)
# print(f"Les joueurs de l'équipe {correspondance_equipes.get(equipe_trouvee)} sont les suivants :")
# print(joueurs1['NAME'])

def cinq_majeur(dataframe, joueurs):
    # Trier les joueurs par ordre décroissant de temps de jeu moyen (MPG)
    equipe_triee = joueurs.sort_values(by='MPG', ascending=False)
    # Sélectionner les cinq premiers joueurs avec le temps de jeu moyen le plus élevé
    cinq_majeur = equipe_triee.head(5)
    return cinq_majeur
cinq_joueurs = cinq_majeur(team_df, joueurs1)
print(f"Les cinq joueurs ayant le temps de jeu moyen le plus élevé dans l'équipe {correspondance_equipes.get(equipe_trouvee)} sont :")
print(cinq_joueurs[['NAME', 'MPG']])

Les cinq joueurs ayant le temps de jeu moyen le plus élevé dans l'équipe Denver Nuggets sont :
                         NAME   MPG
27               Nikola Jokic  33.7
58               Jamal Murray  32.8
175  Kentavious Caldwell-Pope  31.3
87               Aaron Gordon  30.3
74         Michael Porter Jr.  29.0


On répète l'opération pour obtenir une deuxième équipe à laquelle s'opposera la première.

In [67]:
equipes_disponibles = liste_equipes(team_df)
equipe2 = input("Entrez le nom de l'équipe adverse : ")
equipe_trouvee = trouver_equipe(equipe2, equipes_disponibles)
joueurs2 = joueurs_par_equipe(team_df, equipe_trouvee)

cinq_joueurs = cinq_majeur(team_df, joueurs2)
print(f"Les cinq joueurs ayant le temps de jeu moyen le plus élevé dans l'équipe {correspondance_equipes.get(equipe_trouvee)} sont :")
print(cinq_joueurs[['NAME', 'MPG']])

Les cinq joueurs ayant le temps de jeu moyen le plus élevé dans l'équipe New York Knicks sont :
               NAME   MPG
21    Julius Randle  35.5
29    Jalen Brunson  35.0
61       RJ Barrett  33.9
194       Josh Hart  30.0
163  Quentin Grimes  29.9


___

#### Calcul du gamescore

Pour comparer les équipes, nous allons nous appuyer sur les performances individuelles de chaque joueur composant le cinq majeur. Ces performances seront évaluées à l'aide du gamescore. Ce dernier défini une statistique globale définie par la formule suivante, établie par John Hollinger:

Gamescore = Points marqués + (0,4 x Paniers marqués) – (0,7 x Paniers tentés) – (0,4 x (Lancers francs tentés – Lancers francs marqués)) + (0,7 x Rebonds offensifs) + (0,3 x Rebonds défensifs) + Interceptions + (0,7 x Passes décisives) + (0,7 x Contres) – (0,4 x Fautes personnelles) – Turnovers

Dans notre cas, le jeu de données utilisé précédemment n'est pas suffisant, c'est pourquoi nous utilisons un second jeu de données, issu directement du site officiel de la NBA et réalisé par nos soins (scrapping).

In [68]:
def calculate_game_score(pts, fg, fga, ftm, fta, orb, drb, stl, ast, blk, pf, tov):
    game_score = (
        pts + 0.4 * fg + 0.7 * orb + 0.3 * drb + stl + 0.7 * ast + 0.7 * blk
        - 0.7 * fga - 0.4 * (fta-ftm) - 0.4 * pf - tov
    )
    return game_score


def game_score_joueur(dataframe, dataframe2, nom_joueur):
    # Récupérer les données du joueur spécifié
    joueur = dataframe[dataframe['NAME'] == nom_joueur]
    joueur_bis = dataframe2[dataframe2['Player'] == nom_joueur]
    if joueur.empty or joueur_bis.empty:
        print("Le joueur spécifié n'a pas été trouvé.")
        return None
    
    # Convertir les valeurs pertinentes en float ou int
    pts = float(joueur_bis['PTS'].iloc[0])
    fga = float(joueur_bis['FGA'].iloc[0])
    fg_percent = float(joueur_bis['FG%'].iloc[0])/100
    ftm = float(joueur_bis['FTM'].iloc[0])
    fta = float(joueur_bis['FTA'].iloc[0])
    ft_percent = float(joueur_bis['FT%'].iloc[0])/100
    orb = float(joueur_bis['OREB'].iloc[0])
    drb = float(joueur_bis['DREB'].iloc[0])
    stl = float(joueur_bis['STL'].iloc[0])
    ast = float(joueur_bis['AST'].iloc[0])
    blk = float(joueur_bis['BLK'].iloc[0])
    pf = float(joueur_bis['PF'].iloc[0])
    tov = float(joueur_bis['TOV'].iloc[0])
    fg = fg_percent*fga
    gp = float(joueur_bis['GP'].iloc[0])

    # Calcul du Game Score
    game_score = calculate_game_score(pts, fg, fga, ftm, fta, orb, drb, stl, ast, blk, pf, tov)/gp
    return game_score

# Chemin vers le fichier CSV
chemin = "NBA Stats official 2022-3.csv"
# Importer les données depuis le fichier CSV
donnees_bis = pd.read_csv(chemin)

#### Comparaison des gamescores des 5 majeurs des 2 équipes

In [69]:
def cinq_majeur(dataframe, equipe):
    equipe_triee = equipe.sort_values(by='MPG', ascending=False)
    cinq_majeur = equipe_triee.head(5)
    return cinq_majeur

cinq_majeur_equipe1 = cinq_majeur(team, joueurs1)
cinq_majeur_equipe2 = cinq_majeur(team, joueurs2)

noms_joueurs_equipe1 = cinq_majeur_equipe1['NAME'].tolist()
noms_joueurs_equipe2 = cinq_majeur_equipe2['NAME'].tolist()

game_scores_equipe1 = [game_score_joueur(team, donnees_bis, joueur) for joueur in noms_joueurs_equipe1]
game_scores_equipe2 = [game_score_joueur(team, donnees_bis, joueur) for joueur in noms_joueurs_equipe2]

trace_equipe1 = go.Bar(
    x=noms_joueurs_equipe1,
    y=game_scores_equipe1,
    name=equipe1
)

trace_equipe2 = go.Bar(
    x=noms_joueurs_equipe2,
    y=game_scores_equipe2,
    name=equipe2
)

fig = go.Figure(data=[trace_equipe1, trace_equipe2])
fig.update_layout(
    title='Comparaison des Game Scores des cinq majeurs des deux équipes',
    xaxis=dict(title='Joueurs'),
    yaxis=dict(title='Game Score'),
    barmode='group'
)

fig.show()

On a ainsi un premier graphique qui peut nous indiquer des rapports de force entre différentes équipes. Attention, le diagramme affiché ne fait apparaître que le cinq majeur de chaque équipe et les écarts de force peuvent être compensés par de bons joueurs sortant du banc. Cependant, cette simplification nous permet d'obtenir des chances de victoire et des comparaisons plus accessibles.

___
#### Estimation des chances de victoires

In [70]:
score_equipe1 = sum(game_scores_equipe1)
print("La somme des Game Scores des joueurs de l'équipe 1 est :", score_equipe1)

score_equipe2 = sum(game_scores_equipe2)
print("La somme des Game Scores des joueurs de l'équipe 2 est :", score_equipe2)

# Calcul très simplifié d'un pourcentage de victoire possible : 
victoire = 100 * score_equipe1 / (score_equipe1 + score_equipe2) 
# Conserver seulement les quatre premiers chiffres 
vic = round(victoire, 2)
print("L'équipe 1 a un pourcentage de victoire face à l'équipe", correspondance_equipes.get(equipe_trouvee), "de", vic, "%.")


La somme des Game Scores des joueurs de l'équipe 1 est : 77.43439637152193
La somme des Game Scores des joueurs de l'équipe 2 est : 67.6320891075286
L'équipe 1 a un pourcentage de victoire face à l'équipe New York Knicks de 53.38 %.


#### Estimation de l'issue du Best Of 7

A la fin de la saison régulière, il y a en NBA les Playoffs qui sont des affrontements en BO7 (Best Of 7) et qui permettent de déterminer le champion de la NBA. Avec le pourcentage de victoire calculé on peut estimer les probabilités de chaque issue du BO7.

In [71]:
import math

# Probabilité de victoire de l'équipe 1 (en pourcentage)
proba = vic / 100

# Fonction pour calculer le coefficient binomial (combinaison)
def binomial_coefficient(n, k):
    return math.comb(n, k)

# Résultats possibles du BO7
resultats1 = [(4, 3-i) for i in range(4)]
resultats2 = [(i, 4) for i in range(4)]

# Calcul des probabilités de chaque résultat possible pour l'équipe 1
proba1 = {}
for equipe1_wins, equipe2_wins in resultats1:
    prob = binomial_coefficient(equipe1_wins+equipe2_wins, equipe2_wins) * ((1-proba) ** equipe2_wins) * ((proba) ** (equipe1_wins))
    proba1[(equipe1_wins, equipe2_wins)] = prob

# Calcul des probabilités de chaque résultat possible pour l'équipe 2
proba2 = {}
for equipe1_wins, equipe2_wins in resultats2:
    prob = binomial_coefficient(equipe1_wins+equipe2_wins, equipe1_wins) * ((proba) ** equipe1_wins) * ((1-proba) ** (equipe2_wins))
    proba2[(equipe1_wins, equipe2_wins)] = prob

# Affichage des probabilités de chaque résultat possible
for resultat, prob in proba2.items():
    print(f"Probabilité que l'équipe 1 perde {resultat[0]}-{resultat[1]} : {prob:.4f}")
for resultat, prob in proba1.items():
    print(f"Probabilité que l'équipe 1 gagne {resultat[0]}-{resultat[1]} : {prob:.4f}")

Probabilité que l'équipe 1 perde 0-4 : 0.0472
Probabilité que l'équipe 1 perde 1-4 : 0.1261
Probabilité que l'équipe 1 perde 2-4 : 0.2019
Probabilité que l'équipe 1 perde 3-4 : 0.2515
Probabilité que l'équipe 1 gagne 4-3 : 0.2879
Probabilité que l'équipe 1 gagne 4-2 : 0.2647
Probabilité que l'équipe 1 gagne 4-1 : 0.1893
Probabilité que l'équipe 1 gagne 4-0 : 0.0812


On remarque que la somme des probabilités calculées est supérieure à 1. Cela est lié à la manière dont on a calculé ces probabilités, en séparant le calcul en deux selon si l'équipe 1 gagne ou perd. Cependant, les rapports entre les probabilités sont bons et on peut donc normaliser les probabilités obtenues en les divisant par la somme des probabilités.

In [72]:
#On calcule la somme des probabilités
somme =0
for resultat, prob in proba1.items():
    somme +=prob
for resultat, prob in proba2.items():
    somme +=prob
print(somme)

#On normalise les probabilités obtenues et on les ordonne dans une liste
proba = []
for resultat, prob in proba2.items():
    proba.append(prob/somme)
for resultat, prob in proba1.items():
    proba.append(prob/somme)

print(proba)

1.4497765797672457
[0.03258276956904446, 0.08696341197977968, 0.1392632079444192, 0.17345696760170562, 0.19860859996952054, 0.18257823126449765, 0.13054356589768173, 0.05600324577335125]


In [73]:
# Définition des résultats possibles
resultats = ["0-4", "1-4", "2-4", "3-4", "4-3", "4-2", "4-1", "4-0"]

# Création du graphique en barres
fig = go.Figure(go.Bar(x=resultats, y=proba))

# Personnalisation du graphique
fig.update_layout(
    title="Probabilités des issues possibles dans un BO7",
    xaxis_title="Résultat de la série",
    yaxis_title="Probabilité",
    yaxis_tickformat=".2%",
)

# Affichage du graphique
fig.show()


Il pourrait être intéressant de comparer les résultats ainsi obtenus aux résultats des play-offs de l'année dernière, peut-être en donnant des pourcentages de chance de remporter le titre pour chaque équipe.